In [45]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

In [46]:
years =  list(range(2014, 2022 + 1))

In [47]:
races = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'country': [],
        'lat': [],
        'long': [],
        'url': []
        }

for year in years:
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        ### Preventing Abu Dhabi 2022 from being collected as data is not yet available
        if year == 2022:
                if int(item['round']) == 22:
                        break

        races['season'].append(int(item['season']))
        races['round'].append(int(item['round']))
        races['circuit_id'].append(item['Circuit']['circuitId'])
        races['country'].append(item['Circuit']['Location']['country'])    
        races['lat'].append(float(item['Circuit']['Location']['lat']))
        races['long'].append(float(item['Circuit']['Location']['long']))
        races['url'].append(item['url'])

races = pd.DataFrame(races)
races.head()

,season,round,circuit_id,country,lat,long,url
0,2014,1,albert_park,Australia,-37.84970,144.96800,http://en.wikipedia.org/wiki/2014_Australian_G...
1,2014,2,sepang,Malaysia,2.76083,101.73800,http://en.wikipedia.org/wiki/2014_Malaysian_Gr...
2,2014,3,bahrain,Bahrain,26.03250,50.51060,http://en.wikipedia.org/wiki/2014_Bahrain_Gran...
3,2014,4,shanghai,China,31.33890,121.22000,http://en.wikipedia.org/wiki/2014_Chinese_Gran...
4,2014,5,catalunya,Spain,41.57000,2.26111,http://en.wikipedia.org/wiki/2014_Spanish_Gran...


In [48]:
### Query API

schedules = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'date': []
        }

for year in years:

    url = 'http://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        ### Preventing Abu Dhabi 2022 from being collected as data is not yet available
        if year == 2022:
                if int(item['round']) == 22:
                        break

        schedules['season'].append(int(item['season']))
        schedules['round'].append(int(item['round']))
        schedules['circuit_id'].append(item['Circuit']['circuitId'])
        schedules['date'].append(datetime.strptime(item['date']+ 'T' +item['time'], '%Y-%m-%dT%H:%M:%SZ'))

schedules = pd.DataFrame(schedules)
schedules.head()

,season,round,circuit_id,date
0,2014,1,albert_park,2014-03-16 06:00:00
1,2014,2,sepang,2014-03-30 08:00:00
2,2014,3,bahrain,2014-04-06 15:00:00
3,2014,4,shanghai,2014-04-20 07:00:00
4,2014,5,catalunya,2014-05-11 12:00:00


In [49]:
race_schedules = pd.merge(races, schedules, how='inner', on=['season', 'round', 'circuit_id'])

race_schedules.to_csv(path+'data/races.csv', index=False)

In [50]:
races = pd.read_csv(path+'data/races.csv')

rounds = []
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

In [51]:
# query API
    
results = {
    'season': [],
    'round':[],
    'circuit_id':[],
    'driver': [],
    'date_of_birth': [],
    'nationality': [],
    'constructor': [],
    'grid': [],
    'time': [],
    'status': [],
    'points': [],
    'podium': [],
    'fastest_lap': []
    }

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:

            results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            results['driver'].append(item['Driver']['driverId'])
            results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            results['nationality'].append(item['Driver']['nationality'])
            results['constructor'].append(item['Constructor']['constructorId'])
            results['grid'].append(int(item['grid']))

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            results['status'].append(item['status'])
            results['points'].append(float(item['points']))
            results['podium'].append(int(item['position']))

            try:
                results['fastest_lap'].append(item['FastestLap']['Time']['time'])
            except:
                results['fastest_lap'].append(None)


results = pd.DataFrame(results)
results.head()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,fastest_lap
0,2014,1,albert_park,rosberg,1985-06-27,German,mercedes,3,5578710.0,Finished,25.0,1,1:32.478
1,2014,1,albert_park,kevin_magnussen,1992-10-05,Danish,mclaren,4,5605487.0,Finished,18.0,2,1:33.066
2,2014,1,albert_park,button,1980-01-19,British,mclaren,10,5608737.0,Finished,15.0,3,1:32.917
3,2014,1,albert_park,alonso,1981-07-29,Spanish,ferrari,5,5613994.0,Finished,12.0,4,1:33.186
4,2014,1,albert_park,bottas,1989-08-28,Finnish,williams,15,5626349.0,Finished,10.0,5,1:32.616


In [52]:
results.to_csv(path+'data/results.csv', index=False)

In [53]:
driver_standings = {
    'season': [],
    'round':[],
    'driver': [],
    'driver_points': [],
    'driver_wins': [],
    'driver_standings_pos': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))    
            driver_standings['driver'].append(item['Driver']['driverId'])
            driver_standings['driver_points'].append(float(item['points']))
            driver_standings['driver_wins'].append(int(item['wins']))
            driver_standings['driver_standings_pos'].append(int(item['position']))

driver_standings = pd.DataFrame(driver_standings)
driver_standings.tail(21)

,season,round,driver,driver_points,driver_wins,driver_standings_pos
3846,2022,21,leclerc,290.0,3,2
3847,2022,21,perez,290.0,2,3
3848,2022,21,russell,265.0,1,4
3849,2022,21,hamilton,240.0,0,5
3850,2022,21,sainz,234.0,1,6
3851,2022,21,norris,113.0,0,7
3852,2022,21,ocon,86.0,0,8
3853,2022,21,alonso,81.0,0,9
3854,2022,21,bottas,49.0,0,10
3855,2022,21,vettel,36.0,0,11


In [54]:
driver_standings.to_csv(path+'data/driver_standings.csv', index=False)

In [55]:
constructor_rounds = rounds

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []
                   }

for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            constructor_standings['constructor_points'].append(float(item['points']))
            constructor_standings['constructor_wins'].append(int(item['wins']))
            constructor_standings['constructor_standings_pos'].append(int(item['position']))
            
constructor_standings = pd.DataFrame(constructor_standings)
constructor_standings.tail(10)

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
1839,2022,21,red_bull,719.0,16,1
1840,2022,21,ferrari,524.0,4,2
1841,2022,21,mercedes,505.0,1,3
1842,2022,21,alpine,167.0,0,4
1843,2022,21,mclaren,148.0,0,5
1844,2022,21,alfa,55.0,0,6
1845,2022,21,aston_martin,50.0,0,7
1846,2022,21,haas,37.0,0,8
1847,2022,21,alphatauri,35.0,0,9
1848,2022,21,williams,8.0,0,10


In [56]:
constructor_standings.to_csv(path+'data/constructor_standings.csv', index=False)